In [91]:
import pandas as pd
import requests as re
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
load_dotenv()

True

### Create CSV-Files for the Database

#### 1. Customer Data

In [68]:
# CAUTION! Running this cell can take a while!

path = '../datalake/online_retail_II.xlsx'
xl = pd.ExcelFile(path)
# xl.sheet_names  # see all sheet names

df_1 = pd.read_excel(path, sheet_name='Year 2009-2010')
df_2 = pd.read_excel(path, sheet_name='Year 2010-2011')
df = pd.concat([df_1, df_2], axis=0, join='inner', ignore_index=True)
df_copy = df.copy()

# drop all rows with NaNs
df = df.dropna()

In [69]:
# create customer dataset from online retail data
df_customers_pivot = df.pivot_table(index="Customer ID", values="Country", aggfunc=(lambda x: x))
df_customers = pd.DataFrame({'customer_id': df_customers_pivot.index, 'country': df_customers_pivot['Country']}).reset_index()
df_customers['country'] = df_customers['country'].apply(lambda x: x[0].upper())
df_customers['country'] = df_customers['country'].apply(lambda x: 'UNITED KINGDOM' if x=='U' else x)
df_customers['country'] = df_customers['country'].apply(lambda x: 'IRELAND' if x=='EIRE' else x)
df_customers = df_customers[['customer_id', 'country']]
df_customers['customer_id'] = df_customers['customer_id'].astype(int)
df_customers

,customer_id,country
0,12346,UNITED KINGDOM
1,12347,ICELAND
2,12348,FINLAND
3,12349,ITALY
4,12350,NORWAY
...,...,...
5937,18283,UNITED KINGDOM
5938,18284,UNITED KINGDOM
5939,18285,UNITED KINGDOM
5940,18286,UNITED KINGDOM


In [70]:
# save customers table as csv-file
customer_path = "../csv-files/customers_data.csv"
df_customers.to_csv(customer_path, index=False)

### 2. Orders Data

In [71]:
# create orders dataset from online retail data
df_orders = df[['Invoice', 'StockCode', 'Quantity', 'InvoiceDate', 'Customer ID']]
df_orders = df_orders.rename(columns={'Invoice': 'invoice_number', 'StockCode': 'item_id',
                          'Quantity': 'quantity', 'InvoiceDate': 'invoice_date',
                          'Customer ID': 'customer_id'})
df_orders['invoice_date'] = df_orders['invoice_date'].apply(lambda x: int(x.timestamp()))
df_orders['customer_id'] = df_orders['customer_id'].astype(int)
df_orders

,invoice_number,item_id,quantity,invoice_date,customer_id
0,489434,85048,12,1259653500,13085
1,489434,79323P,12,1259653500,13085
2,489434,79323W,12,1259653500,13085
3,489434,22041,48,1259653500,13085
4,489434,21232,24,1259653500,13085
...,...,...,...,...,...
1067366,581587,22899,6,1323435000,12680
1067367,581587,23254,4,1323435000,12680
1067368,581587,23255,4,1323435000,12680
1067369,581587,22138,3,1323435000,12680


In [72]:
# save orders table as csv-file
orders_path = "../csv-files/orders_data.csv"
df_orders.to_csv(orders_path, index=False)

### 3. Items Data

In [73]:
# create itmes dataset from online retail data
df_items_group = df.groupby('StockCode').agg({'Description': lambda x: ', '.join(x.unique())})
df_description = pd.DataFrame({'Description': df_items_group['Description']}).reset_index()
df_items = pd.merge(df_description, df[['StockCode', 'Price']], on="StockCode", how='left').drop_duplicates(subset=['StockCode'])
df_items = df_items.rename(columns={'StockCode': 'item_id', 'Price': 'price_in_GBP', 'Description': 'description'})
df_items

,item_id,description,price_in_GBP
0,10002,INFLATABLE POLITICAL GLOBE,0.850
324,10080,GROOVY CACTUS INFLATABLE,0.850
351,10109,BENDY COLOUR PENCILS,0.420
352,10120,DOGGY RUBBER,0.210
419,10125,MINI FUNKY DESIGN TAPES,0.850
...,...,...,...
822308,PADS,PADS TO MATCH ALL CUSHIONS,0.001
822327,POST,POSTAGE,18.000
824346,SP1002,KID'S CHALKBOARD/EASEL,2.950
824348,TEST001,This is a test product.,4.500


In [74]:
# save items table as csv-file
items_path = "../csv-files/items_data.csv"
df_items.to_csv(items_path, index=False)

#### 4. Currency Exchange Rate Data

In [75]:
# get list of currency names for currencies from API
API_KEY = os.getenv('api_key')
request_url = f"https://v6.exchangerate-api.com/v6/{API_KEY}/codes"
response = re.get(url=request_url)

currency_codes = [result[0] for result in response.json()['supported_codes']]
currency_names = [result[1] for result in response.json()['supported_codes']]

df_codes = pd.DataFrame({'currency_code': currency_codes, 'currency_name': currency_names})

In [76]:
h# get latest exchange rates for 'GBP' (Pound Sterlin) and latest update value
CURRENCY_CODE = 'GBP'
request_url = f"https://v6.exchangerate-api.com/v6/{API_KEY}/latest/{CURRENCY_CODE}"
response = re.get(url=request_url)

rate_dict = response.json()['conversion_rates']
currencies = [currency for currency in rate_dict.keys()]
exchange_rates = [exchange_rate for exchange_rate in rate_dict.values()]

date = response.json()['time_last_update_unix']
currency_dates = [date for k in range(len(exchange_rates))]

df_exchange = pd.DataFrame({'currency_code': currencies, 'exchange_rate': exchange_rates, 'last_update': currency_dates})

In [77]:
# merge currency names and exchange rates to create currencies table
df_currencies = pd.merge(df_codes, df_exchange, on='currency_code', how='outer')
df_currencies

,currency_code,currency_name,exchange_rate,last_update
0,AED,UAE Dirham,4.740978,1721426401
1,AFN,Afghan Afghani,91.978092,1721426401
2,ALL,Albanian Lek,119.297299,1721426401
3,AMD,Armenian Dram,501.614582,1721426401
4,ANG,Netherlands Antillian Guilder,2.310783,1721426401
...,...,...,...,...
157,XPF,CFP Franc,141.598874,1721426401
158,YER,Yemeni Rial,325.318848,1721426401
159,ZAR,South African Rand,23.603420,1721426401
160,ZMW,Zambian Kwacha,33.279383,1721426401


In [78]:
# save currencies table as csv-file
currencies_path = "../csv-files/currency_exchange_data.csv"
df_currencies.to_csv(currencies_path, index=False)

#### 5. Country Courrency Data

In [79]:
# retrieve data about countries and their currencies from website 
request_url = "https://www.iban.com/currency-codes"
response = re.get(request_url)

soup = BeautifulSoup(response.content, "html.parser")

result = soup.find(name='table', class_='table table-bordered downloads tablesorter')
items = result.find_all(name='tr')

In [80]:
# create countries table
country_names = []
currency_codes = []
for k in range(len(items)-1):
    k += 1
    item = str(items[k]).split('</td>')
    country_name = item[0][9:]
    currency_code = item[2][-3:]
    country_names.append(country_name)
    currency_codes.append(currency_code)
df_countries_0 = pd.DataFrame({'country': country_names, 'currency_code': currency_codes})
df_countries_group = df_countries_0.groupby('country').agg({'currency_code': lambda x: ', '.join(x.unique())})
df_countries = pd.DataFrame({'currency_code': df_countries_group['currency_code']}).reset_index()
df_countries['country'] = df_countries['country'].apply(lambda x: 'UNITED KINGDOM' if x=='UNITED KINGDOM OF GREAT    BRITAIN AND NORTHERN IRELAND (THE)' else x)
df_countries['country'] = df_countries['country'].apply(lambda x: 'UNITED ARAB EMIRATES' if x=='UNITED ARAB EMIRATES (THE)' else x)
df_countries['country'] = df_countries['country'].apply(lambda x: 'NETHERLANDS' if x=='NETHERLANDS (THE)' else x)
df_countries['currency_code'] = df_countries['currency_code'].apply(lambda x: 'CHF' if x=='CHE, CHF, CHW' else x)

In [81]:
# save countries table as csv-file
countries_path = "../csv-files/currency_country_data.csv"
df_countries.to_csv(countries_path, index=False)

### DATAFRAMES OVERVIEW

In [82]:
df_customers.head(3)

,customer_id,country
0,12346,UNITED KINGDOM
1,12347,ICELAND
2,12348,FINLAND


In [83]:
df_orders.head()

,invoice_number,item_id,quantity,invoice_date,customer_id
0,489434,85048,12,1259653500,13085
1,489434,79323P,12,1259653500,13085
2,489434,79323W,12,1259653500,13085
3,489434,22041,48,1259653500,13085
4,489434,21232,24,1259653500,13085


In [84]:
df_items.head(3)

,item_id,description,price_in_GBP
0,10002,INFLATABLE POLITICAL GLOBE,0.85
324,10080,GROOVY CACTUS INFLATABLE,0.85
351,10109,BENDY COLOUR PENCILS,0.42


In [85]:
df_currencies.head(3)

,currency_code,currency_name,exchange_rate,last_update
0,AED,UAE Dirham,4.740978,1721426401
1,AFN,Afghan Afghani,91.978092,1721426401
2,ALL,Albanian Lek,119.297299,1721426401


In [103]:
# Make sure all the values of reference key get included in foreign key of countries table 
common_df_countries = df_countries[df_countries['currency_code'].isin(df_currencies['currency_code'])]


142

In [126]:
# Make sure all the values of reference key get included in foreign key of customers table 
df_customers_updated = df_customers.merge(common_df_countries[['country']], on='country', how='inner')


In [137]:
# Make sure all the values of reference key get included in foreign key of order table 
df_orders_updated = df_orders.merge(df_customers_updated[['customer_id']], on='customer_id', how='inner')


(821302, 5)

### POPULATING TABLES IN MYSQL SCHEMA (retail_online) FROM DATABASES

In [93]:


user = "root"
password_ = os.getenv('SECRET_KEY')
host = "127.0.0.1"
port = "3306"
db_actions = "retail_online"
from sqlalchemy import create_engine
engine = create_engine(f'mysql+mysqlconnector://{user}:{password_}@{host}:{port}/{db_actions}')

In [115]:
# populate currencies table
df_currencies.to_sql(name='currencies', con=engine, index=False, if_exists='append')

162

In [116]:
# populate countries table
common_df_countries.to_sql(name='countries', con=engine, index=False, if_exists='append')

233

In [128]:
# populate customers table
df_customers_updated.to_sql(name='customers', con=engine, index=False, if_exists='append')

5897

In [130]:
# populate items table
df_items.to_sql(name='items', con=engine, index=False, if_exists='append')

4646

In [138]:
# populate orders table
df_orders_updated.to_sql(name='orders', con=engine, index=False, if_exists='append')

821302